# Video Converter Notebook
This notebook contains the code needed to convert a greyscale video to a quadtree-compressed video.

The video should be split into individual frames for processing. This can be  done with ffmpeg:
```bash
ffmpeg -i ".\Touhou - Bad Apple.mp4" "media\badapple%04d.png"
```

This notebook saves the compressed frames as a pickled python object (a tuple of two numpy arrays: the animation (a 4-component 2d array of rectangle definitions (value, x, y, subdivision)), and the array of frame lengths (each frame uses a variable number of rectangles to represent it)).

This is inteneded to be used by the pySSV video playback demo [here](https://github.com/space928/Shaders-For-Scientific-Visualisation/tree/main/examples).

In [ ]:
%pip install multiprocess
%pip install tqdm
%pip install pillow
%pip install numpy

In [8]:
# Video configuration
min_frame = 1
max_frame = 6569
name_fmt = "media/badapple{0:04d}.png"
files = [name_fmt.format(x) for x in range(min_frame, max_frame+1)]
tolerance = 50  # Maximum amount of deviation (0-255) in a cell to be considered 'uniform' in colour
max_div = 7  # Maximum number of quadtree subdivisions, this effectively controls the resolution (resolution=2**max_div)

In [12]:
from PIL import Image
import numpy as np
import pickle as pkl
from multiprocess import Pool
import tqdm

def print_img(data, img):
    div = 8
    for y in range(0, img.height, div):
        print(' '.join('#' if data[y, x] > 128 else ' ' for x in range(0, img.width, div)))

def init_pool_proc(_tolerance, _max_div):
    # Jank to make the multiprocessing work in notebooks
    global tolerance
    global max_div
    tolerance = _tolerance
    max_div = _max_div

    global Image
    global np
    from PIL import Image
    import numpy as np

    global check_colour
    def check_colour(data):
        # Check if a given cell is uniform in colour; returns true if the cell is uniform => no more subdivision needed
        col0 = data[0]
        return np.all(np.abs(data - col0) < tolerance)

    global divide_frame
    def divide_frame(data, x=0, y=0, it=0):
        # Recursively subdivides the frame into quads, returning the list of rectangles which make up the frame
        w, h = data.shape
        if it < max_div and (it==0 or not check_colour(data)):
            rects = []
            rects.extend(divide_frame(data[:w//2,:h//2], x+0,    y+0, it+1))
            rects.extend(divide_frame(data[w//2:,:h//2], x+w//2, y+0, it+1))
            rects.extend(divide_frame(data[:w//2,h//2:], x+0,    y+h//2, it+1))
            rects.extend(divide_frame(data[w//2:,h//2:], x+w//2, y+h//2, it+1))
            return rects
        return [(data[w//2,h//2], x, y, it)]

init_pool_proc(tolerance, max_div)

def proc_frame(img_path):
    img = Image.open(img_path)
    data = np.array(img.getdata())
    # print(img.width, img.height, img.mode)
    data = data.reshape((img.height, img.width, len(img.mode)))
    data = data[:, :, 1]
    # print_img(data, img)
    encoded = np.array(divide_frame(data), dtype=np.int16)
    # print(encoded)
    # print(encoded.shape)
    
    return encoded

def gen_texture():
    print(f"Processing {len(files)} frames...")
    with Pool(16, initializer=init_pool_proc, initargs=(tolerance, max_div)) as pool:
        frames = list(tqdm.tqdm(pool.imap(proc_frame, files, chunksize=20), total=len(files)))

    print(f"Concatenating frames...")
    max_width = np.max([frame.shape[0] for frame in frames])  # 4**(max_div)
    tex = np.zeros((max_width, max_frame, 4), dtype=np.int16)
    frame_lengths = np.zeros(max_frame, dtype=np.int16)
    
    for i in tqdm.trange(len(files)):
        frame = frames[i]
        # frame = proc_frame(files[i])
        frame_lengths[i] = frame.shape[0]
        tex[:frame.shape[0], i, :] = frame
    return tex, frame_lengths
    
# proc_frame(files[100])
tex = gen_texture()

Processing 6569 frames...


100%|██████████| 6569/6569 [01:09<00:00, 93.92it/s] 


Concatenating frames...


100%|██████████| 6569/6569 [00:00<00:00, 59597.20it/s]


In [13]:
anim, f_len = tex
print(anim.dtype, anim.shape)
print(f_len.dtype, f_len.shape)

# print(anim)

# Save the frames to a pickled blob
with open("badapple_quad.pkl", "wb") as f:
    pkl.dump(tex, f)

int16 (4603, 6569, 4)
int16 (6569,)
[[[  0   0   0   1]
  [  0   0   0   1]
  [  0   0   0   1]
  ...
  [  0   0   0   1]
  [  0   0   0   1]
  [  0   0   0   1]]

 [[  0 180   0   1]
  [  0 180   0   1]
  [  0 180   0   1]
  ...
  [  0 180   0   1]
  [  0 180   0   1]
  [  0 180   0   1]]

 [[  0   0 240   1]
  [  0   0 240   1]
  [  0   0 240   1]
  ...
  [  0   0 240   1]
  [  0   0 240   1]
  [  0   0 240   1]]

 ...

 [[  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]
  ...
  [  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]]

 [[  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]
  ...
  [  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]]

 [[  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]
  ...
  [  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]]]
